In [1]:
import pandas as pd
from elasticsearch import Elasticsearch

In [9]:
es = Elasticsearch('192.168.50.111:19200')

In [10]:
es.ping()

True

## Elasticsearch利用scroll查询获取所有数据

In [12]:
body = {
    'query': {
        'bool': {
            'must': [{'term': {'metric_table': 'service_relation_client_resp_time'}}]
        }
    }
}
queryData = es.search(index = 'dc_service_metrics_avg_20210916', scroll='5m', timeout='60s', size=100, body=body)

In [13]:
metadata = queryData.get("hits").get("hits")

In [16]:
len(metadata)

100

In [17]:
scroll_id = queryData["_scroll_id"]

In [18]:
total = queryData["hits"]["total"]

In [22]:
for i in range(int(total / 100)):
    res = es.scroll(scroll_id=scroll_id, scroll='5m')  
    metadata += res["hits"]["hits"]

In [23]:
data = pd.DataFrame([x['_source'] for x in metadata])

In [24]:
data

,metric_table,count,entity_id,value,timestamp,summation
0,service_relation_client_resp_time,6,YXBpLWtleXxkY2dsOjpzY3JlZW4=-YXBpLWtleXxteXNxbA==,4,1631721600,24
1,service_relation_client_resp_time,5,YXBpLWtleXxkY2dsOjpzdGFuZGFyZA==-YXBpLWtleXxkY...,55,1631721600,276
2,service_relation_client_resp_time,1,dGVzdC1hcGkta2V5LXNlbmQtY2hlY2t8ZGF0YWJ1ZmY6On...,1,1631721600,1
3,service_relation_client_resp_time,2,MTkyMTY4NTAzMTEwN3xsb2djdWJlOjpsb2djdWJlX3Njcm...,5,1631721600,10
4,service_relation_client_resp_time,2,dGVzdC1hcGkta2V5LXNlbmQtY2hlY2t8ZGF0YWJ1ZmY6On...,0,1631721600,0
...,...,...,...,...,...,...
16122,service_relation_client_resp_time,380,dGVzdC1hcGkta2V5LXNlbmQtY2hlY2t8ZGF0YWJ1ZmY6On...,0,1631807940,8
16123,service_relation_client_resp_time,2,YXBpLWtleXxkY2dsOjpzY3JlZW4=-YXBpLWtleXxlbGFzd...,14,1631807940,29
16124,service_relation_client_resp_time,208,dGVzdC1hcGkta2V5LXNlbmQtY2hlY2t8ZGF0YWJ1ZmY6On...,5,1631807940,1111
16125,service_relation_client_resp_time,8,MTkyMTY4NTAzMTEwN3xsb2djdWJlOjpsb2djdWJlX3Njcm...,0,1631807940,0
